<a href="https://colab.research.google.com/github/Abhi0496/CMPE255-Bonus-work/blob/main/Bonuswork1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

Installing openvino and Pillow libraries

In [1]:
!pip install -q openvino-dev[onnx]
!pip install Pillow==6.2.2

     |████████████████████████████████| 5.8 MB 7.8 MB/s 
     |████████████████████████████████| 1.3 MB 40.0 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 7.6 MB 48.7 MB/s 
     |████████████████████████████████| 9.5 MB 42.5 MB/s 
     |████████████████████████████████| 1.9 MB 53.4 MB/s 
     |████████████████████████████████| 13.5 MB 6.9 MB/s 
     |████████████████████████████████| 517 kB 46.6 MB/s 
     |████████████████████████████████| 604 kB 51.2 MB/s 
     |████████████████████████████████| 25.9 MB 1.5 MB/s 
     |████████████████████████████████| 15.6 MB 8.8 MB/s 
     |████████████████████████████████| 26.8 MB 80.0 MB/s 
     |████████████████████████████████| 5.5 MB 29.2 MB/s 
     |████████████████████████████████| 298 kB 54.1 MB/s 
     |████████████████████████████████| 2.0 MB 48.0 MB/s 
     |████████████████████████████████| 3.2 MB 44.4 MB/s 
     |████████████████████████████████| 3.3 MB 43.0 MB/s 
     |████████

In [1]:
import onnx
import torch
import numpy as np
from torch import nn
from torchvision import models
from openvino.runtime import Core

Creating a model with resnet 34 with parameter pretrained=True

In [2]:
model = models.resnet34(pretrained=True)
model

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Creating a random input in tensor format and then converting that input into numpy which will be used further for openvino inference

In [3]:
dummy_input = torch.randn(1, 3, 224, 224)
dummy_np = dummy_input.numpy()

model = model.eval()

Exporting the torch output into resnet34.onnx file(format)

In [4]:
torch.onnx.export(
    model.cpu(),
    dummy_input,
    'resnet34.onnx',
    opset_version=12,
    do_constant_folding=True,
)


onnx_model = onnx.load('resnet34.onnx')
onnx.checker.check_model(onnx_model)


Using the resnet34.onix as input. It will create openvino_model as output which contains- resnet34.xml, resnet34.bin, resnet34.mapping. It also shows the total execution time and memory consumed


In [5]:
!mo --input_model "resnet34.onnx" --input_shape "[1, 3, 224, 224]" --data_type FP16 --output_dir "openvino_model" 

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/content/resnet34.onnx
	- Path for generated IR: 	/content/openvino_model
	- IR output name: 	resnet34
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1, 3, 224, 224]
	- Source layout: 	Not specified
	- Target layout: 	Not specified
	- Layout: 	Not specified
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- User transformations: 	Not specified
	- Reverse input channels: 	False
	- Enable IR generation for fixed input shape: 	False
	- Use the transformations config file: 	None
Advanced parameters:
	- Force the usage of legacy Frontend of Model Optimizer for model conversion into IR: 	False
	- Force the usage of new Frontend of Model Optimizer for model conversi

## Inference Speed
Calculating the inference speed from Pytorch and Openvino.
The input for Pytorch will be in tensor format while for Openvino it will be in numpy format

In [6]:
## Pytorch Inference
%%timeit
with torch.no_grad(): model(dummy_input)

206 ms ± 66.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
## OpenVino Inference
ie = Core()
ir_model = ie.read_model("openvino_model/resnet34.xml")
compiled_model = ie.compile_model(model=ir_model)

In [8]:
%%timeit
compiled_model([dummy_np])

110 ms ± 888 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


As can be seen abovr Pytorch takes twice the time as compared to Openvino

## Comparing Outputs

In [22]:
## Pytorch Output
with torch.no_grad():
  torch_out = model(dummy_input)
torch_out = torch_out.cpu().numpy()[0]


In [23]:
## OpenVINO Output
out_layer = compiled_model.output(0)
openvino_out = compiled_model([dummy_np])[out_layer][0]

Checking the total number of outputs

In [26]:
torch_out.shape, openvino_out.shape

((1000,), (1000,))

Printing the first 5 outputs for Pytorch and Openvino


In [27]:
torch_out[:5], openvino_out[:5]

(array([-0.63275534,  1.2412245 , -0.7959824 , -1.0033318 ,  2.5963533 ],
       dtype=float32),
 array([-0.63309664,  1.244981  , -0.797132  , -1.000312  ,  2.601338  ],
       dtype=float32))

This fucntions does element wise comparison. It returns a boolean array where two arrays are element-wise equal within a tolerance. 1000 output indicates that values are same

In [31]:
np.isclose(torch_out, openvino_out, atol=1e-2).sum()

1000

In [37]:
import numpy
ind_torch = np.argpartition(torch_out, -4)[-4:]

top4_torch = torch_out[ind_torch]
numpy.sort(top4_torch)


array([5.443203 , 5.834096 , 5.9412527, 6.017132 ], dtype=float32)

In [38]:
ind_openvino = np.argpartition(openvino_out, -4)[-4:]

top4_openvino = torch_out[ind_openvino]
numpy.sort(top4_openvino)


array([5.443203 , 5.834096 , 5.9412527, 6.017132 ], dtype=float32)

It can also be seen that the top 4 values are also same for Pytorch and OpenVino